# Парсинг сайта Sokolov

Импортируем библиотеки

In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import re
import time

Объявим константы, которые будем передавать

In [2]:
URL = 'https://sokolov.ru/jewelry-catalog/diamond/'
HEADER = {'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:71.0) Gecko/20100101 Firefox/71.0', 'accept': '*/*'}
HOST = 'https://sokolov.ru/'

Напишем функцию для поиска последней страницы выбранного каталога и создания списка всех страниц каталога

In [3]:
def get_pages(URL, pages_link, url_catalog):
    # сделаем запрос
    r = requests.get(URL, headers = HEADER)
    # применим soup 
    soup = BeautifulSoup(r.text, 'html.parser')
    # найдем класс со страницей
    page = soup.find_all(class_ = 'pagination')
    # получим последнюю страницу
    num_pages = int(max(re.findall(r'page=([\d]+)', str(page))))
    # в итерации получим список ссылок на страницы каталога
    for page in range(1, num_pages+1):
        url = url_catalog + str(page)
        pages_link.append(url)
    return pages_link

Применим и получим список ссылок

In [4]:
url_catalog = 'https://sokolov.ru/jewelry-catalog/diamond/?page='
pages_link = []
get_pages(URL, pages_link, url_catalog)

['https://sokolov.ru/jewelry-catalog/diamond/?page=1',
 'https://sokolov.ru/jewelry-catalog/diamond/?page=2',
 'https://sokolov.ru/jewelry-catalog/diamond/?page=3',
 'https://sokolov.ru/jewelry-catalog/diamond/?page=4',
 'https://sokolov.ru/jewelry-catalog/diamond/?page=5',
 'https://sokolov.ru/jewelry-catalog/diamond/?page=6',
 'https://sokolov.ru/jewelry-catalog/diamond/?page=7',
 'https://sokolov.ru/jewelry-catalog/diamond/?page=8',
 'https://sokolov.ru/jewelry-catalog/diamond/?page=9',
 'https://sokolov.ru/jewelry-catalog/diamond/?page=10',
 'https://sokolov.ru/jewelry-catalog/diamond/?page=11',
 'https://sokolov.ru/jewelry-catalog/diamond/?page=12',
 'https://sokolov.ru/jewelry-catalog/diamond/?page=13',
 'https://sokolov.ru/jewelry-catalog/diamond/?page=14',
 'https://sokolov.ru/jewelry-catalog/diamond/?page=15',
 'https://sokolov.ru/jewelry-catalog/diamond/?page=16',
 'https://sokolov.ru/jewelry-catalog/diamond/?page=17',
 'https://sokolov.ru/jewelry-catalog/diamond/?page=18',
 

In [8]:
jewelry_list = []
for link in pages_link[:2]:
    # Покажем страницу, которую парсим
    print(link)
    # Подождем 1 секунду
    time.sleep(1)
    # сделаем запрос
    r = requests.get(link, headers = HEADER)
    # применим soup
    soup = BeautifulSoup(r.text, 'html.parser')
    # произведем поиск
    page = soup.find_all(class_ = 'product-item-props')
    # в итерации пройдем по списку изделий
    for jewelry in page:
        # подождем 1 секунду
        time.sleep(1)
        # сохраним в переменную наименование 
        name = jewelry.find(class_ = 'product-description').get_text().strip()
        # сохраним в переменную ссылку 
        link = re.findall(r'href=\"/jewelry-catalog/(\S+/)', str(jewelry))
        # сохраним в переменную цену 
        price = jewelry.find(class_ = 'price main').get_text().strip()
        # получим артикул из ссылки
        articul = str(re.findall(r'product/(\d+)', link[0])).replace('[', '').replace(']', '')
        # перейдем по ссылке конкретного изделия
        r_temp = requests.get('https://sokolov.ru/jewelry-catalog/'+link[0], headers = HEADER)
        # применим soup
        soup_temp = BeautifulSoup(r_temp.text, 'html.parser')
        # получим вес и сохраним в переменную
        weight = str(re.findall(r'вес\s+(\d+.\d+)',soup_temp.find(class_ = 'props').get_text().strip())).replace('[', '').replace(']', '')
        # получим описание вставки и сохраним в переменную
        gem = re.findall(r'Вставка\s+(.+)',soup_temp.find(class_ = 'props').get_text().strip())
        # сохраним в писок изделие
        jewelry_list.append([name, articul, price, 'https://sokolov.ru/jewelry-catalog/'+link[0], weight, gem])

https://sokolov.ru/jewelry-catalog/diamond/?page=1
https://sokolov.ru/jewelry-catalog/diamond/?page=2


In [9]:
data_jewel = pd.DataFrame(jewelry_list, columns = ['name', 'articul', 'price', 'link', 'weight', 'gem'])

In [10]:
data_jewel

,name,articul,price,link,weight,gem
0,Серьги из белого золота с бриллиантами,'1021341',27 590 ₽,https://sokolov.ru/jewelry-catalog/product/102...,'1.99',"[Бриллиант (2 шт, 0.054 карат)]"
1,Помолвочное кольцо из белого золота 585 пробы ...,'1011493',19 490 ₽,https://sokolov.ru/jewelry-catalog/product/101...,'1.35',"[Бриллиант (1 шт, 0.027 карат)]"
2,Серьги из белого золота с бриллиантами,'1021314',29 990 ₽,https://sokolov.ru/jewelry-catalog/product/102...,'2.68',"[Бриллиант (16 шт, 0.051 карат), Бриллиант (2 ..."
3,Кольцо из белого золота 585 пробы с бриллиантами,'1011803',27 990 ₽,https://sokolov.ru/jewelry-catalog/product/101...,,"[Бриллиант (20 шт, 0.063 карат), Бриллиант (1 ..."
4,Серьги из белого золота с бриллиантами,'1021316',16 590 ₽,https://sokolov.ru/jewelry-catalog/product/102...,'1.84',"[Бриллиант (10 шт, 0.038 карат)]"
...,...,...,...,...,...,...
139,Помолвочное кольцо из серебра с бриллиантом,'87010002',3 110 ₽,https://sokolov.ru/jewelry-catalog/product/870...,'1.52',"[Бриллиант (1 шт, 0.030 карат)]"
140,Кольцо из серебра с бриллиантом,'87010016',1 070 ₽,https://sokolov.ru/jewelry-catalog/product/870...,'1.11',"[Бриллиант (1 шт, 0.008 карат)]"
141,Помолвочное кольцо из серебра с бриллиантом,'87010018',1 070 ₽,https://sokolov.ru/jewelry-catalog/product/870...,'1.12',"[Бриллиант (1 шт, 0.008 карат)]"
142,Кольцо из серебра с родированием с бриллиантом,'87010057',1 310 ₽,https://sokolov.ru/jewelry-catalog/product/870...,'1.50',[Бриллиант (1 шт)]
